In [69]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime, timedelta, timezone
import requests as rq

In [70]:
df = pd.read_json("./schedule.json")
df['round_old'] = df['round']
df['round'] = df.apply(lambda x: x['round_old'] - 2, axis=1)
df = df.drop([0,1]).drop(columns=['round_old', 'format', 'meetingStartDate']).set_index('round').replace('TBC', None)
df.head(5)

,meetingCountryName,meetingLocation,meetingOfficialName,meetingName,session1,session1Date,session2,session2Date,session3,session3Date,session4,session4Date,session5,session5Date
round,,,,,,,,,,,,,,
1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,Bahrain Grand Prix,Practice 1,2022-03-18T15:00:00,Practice 2,2022-03-18T18:00:00,Practice 3,2022-03-19T15:00:00,Qualifying,2022-03-19T18:00:00,Race,2022-03-20T18:00:00
2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,Saudi Arabian Grand Prix,Practice 1,2022-03-25T17:00:00,Practice 2,2022-03-25T20:00:00,Practice 3,2022-03-26T17:00:00,Qualifying,2022-03-26T20:00:00,Race,2022-03-27T20:00:00
3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,Australian Grand Prix,Practice 1,2022-04-08T13:00:00,Practice 2,2022-04-08T16:00:00,Practice 3,2022-04-09T13:00:00,Qualifying,2022-04-09T16:00:00,Race,2022-04-10T15:00:00
4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DELL'EMILIA ROMAGN...,Emilia Romagna Grand Prix,Practice 1,None,Practice 2,None,Practice 3,None,Qualifying,None,Race,2022-04-24T15:00:00
5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,Miami Grand Prix,Practice 1,2022-05-06T14:30:00,Practice 2,2022-05-06T17:30:00,Practice 3,2022-05-07T13:00:00,Qualifying,2022-05-07T16:00:00,Race,2022-05-08T15:30:00


In [71]:
# gets GMT offset
headers = {"authority": "api.formula1.com","method": "GET","path": "/v1/event-tracker","scheme": "https","accept": "application/json, text/javascript, */*; q=0.01","accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9","apikey": "qPgPPRJyGCIPxFT3el4MF7thXHyJCzAP","locale": "en","origin": "https://www.formula1.com","referer": "https://www.formula1.com/","sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "Windows","sec-fetch-dest": "empty","sec-fetch-mode": "cors","sec-fetch-site": "same-site","user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}
res = rq.get("https://api.formula1.com/v1/editorial-eventlisting/events", headers=headers).json()['events']
dfTz = pd.DataFrame(res)
dfTz = dfTz[['meetingOfficialName', 'gmtOffset']].drop([0,1]).reset_index(drop=True).set_index('meetingOfficialName')
df['gmtOffset'] = df.apply(lambda x: dfTz.at[x['meetingOfficialName'],'gmtOffset'], axis=1)

# changing time str to datetime object
sess = ['Session1Date', '']
for i in range(1, 6):
    df['newSession' + str(i) + 'Date'] = df.apply(lambda x: None if x['session' + str(i) + 'Date'] is None else datetime.strptime(str(x['session' + str(i) + 'Date']) + str(x['gmtOffset']), "%Y-%m-%dT%H:%M:%S%z").astimezone(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ'), axis=1)

#dropping old times
df.drop(columns=['session1Date', 'session2Date', 'session3Date', 'session4Date', 'session5Date'], inplace=True)
df.rename(columns={'newSession1Date': 'session1Date', 'newSession2Date': 'session2Date', 'newSession3Date': 'session3Date', 'newSession4Date': 'session4Date', 'newSession5Date': 'session5Date'}, inplace=True)


df

,meetingCountryName,meetingLocation,meetingOfficialName,meetingName,session1,session2,session3,session4,session5,gmtOffset,session1Date,session2Date,session3Date,session4Date,session5Date
round,,,,,,,,,,,,,,,
1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,Bahrain Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+03:00,2022-03-18T12:00:00Z,2022-03-18T15:00:00Z,2022-03-19T12:00:00Z,2022-03-19T15:00:00Z,2022-03-20T15:00:00Z
2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,Saudi Arabian Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+03:00,2022-03-25T14:00:00Z,2022-03-25T17:00:00Z,2022-03-26T14:00:00Z,2022-03-26T17:00:00Z,2022-03-27T17:00:00Z
3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,Australian Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+10:00,2022-04-08T03:00:00Z,2022-04-08T06:00:00Z,2022-04-09T03:00:00Z,2022-04-09T06:00:00Z,2022-04-10T05:00:00Z
4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DELL'EMILIA ROMAGN...,Emilia Romagna Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+02:00,None,None,None,None,2022-04-24T13:00:00Z
5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,Miami Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,-04:00,2022-05-06T18:30:00Z,2022-05-06T21:30:00Z,2022-05-07T17:00:00Z,2022-05-07T20:00:00Z,2022-05-08T19:30:00Z
6,Spain,Barcelona,FORMULA 1 PIRELLI GRAN PREMIO DE ESPAÑA 2022,Spanish Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+02:00,2022-05-20T12:00:00Z,2022-05-20T15:00:00Z,2022-05-21T11:00:00Z,2022-05-21T14:00:00Z,2022-05-22T13:00:00Z
7,Monaco,Monte Carlo,FORMULA 1 GRAND PRIX DE MONACO 2022,Monaco Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+02:00,2022-05-27T12:00:00Z,2022-05-27T15:00:00Z,2022-05-28T11:00:00Z,2022-05-28T14:00:00Z,2022-05-29T13:00:00Z
8,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2022,Azerbaijan Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,+04:00,2022-06-10T11:00:00Z,2022-06-10T14:00:00Z,2022-06-11T11:00:00Z,2022-06-11T14:00:00Z,2022-06-12T11:00:00Z
9,Canada,Montréal,FORMULA 1 GRAND PRIX DU CANADA 2022,Canadian Grand Prix,Practice 1,Practice 2,Practice 3,Qualifying,Race,-04:00,2022-06-17T18:00:00Z,2022-06-17T21:00:00Z,2022-06-18T17:00:00Z,2022-06-18T20:00:00Z,2022-06-19T18:00:00Z


In [72]:
# saves df as pickle
df.to_pickle('./raceSchedule.pkl')